In [123]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import expanduser, join
from datetime import datetime
from qlabs.business_days import DateManager
from qlabs.database import PostgreSQL

In [124]:
# Le o dataframe com os dados
df = pd.read_excel(r'C:\Users\PedroCervo-Quantitas\Downloads\Atribuicao_rating_proprietario (3).xlsx')

In [ ]:
# Parametros de rating
controlador = {'0':0,'1':-1}
dl_ebitda = {'0':1,'1':2,'2':3,'3':4,'4':5}
margem_ebitda = {'0.1':5,'0.295':4,'0.3':3,'0.4':2,'0.5':1}
fco_ebitda = {'0.05':5,'0.1':4,'0.2':3,'0.4':2,'0.6':1}
ebitda_resultado_financeiro = {'1':5,'1.5':4,'2.5':3,'3':2,'4':1}
caixa_divida_cp = {'0':5,'0.5':4,'1':3,'1.5':2,'2':1}
roe = {'0':5,'0.05':4,'0.1':3,'0.15':2,'0.2':1}

In [126]:
pesos = {'controlador':0.1,'dl_ebitda':0.2,'margem_ebitda':0.15,'fco_ebitda':0.15,'ebitda_resultado_financeiro':0.1,'caixa_divida_cp':0.15,'roe':0.15}

In [127]:
empresa1 = df.iloc[32,:]

In [128]:
empresa1

Data                                                  2023-03-29 00:00:00
Emissor                                                 TELEFONICA BRASIL
Setor                                                             Telecom
Controlador (1=sim, 0=não)                                              1
DL/EBITDA                                                             0.3
Margem EBITDA                                                        0.31
FCO/EBITDA                                                           1.25
EBITDA/Resultado Financeiro                                           8.5
Caixa/Divida CP                                                       0.9
ROE                                                                 0.059
Rating Interno                                                        AAA
Válido                                                                NaN
*Divida liquia/valor da firma; colocar FCF tambem                     NaN
Name: 32, dtype: object

In [129]:
setor_cliente = empresa1['Setor']
controlador_cliente = empresa1['Controlador (1=sim, 0=não)']
dl_ebitda_cliente = empresa1['DL/EBITDA']
margem_ebitda_cliente = empresa1['Margem EBITDA']
fco_ebitda_cliente = empresa1['FCO/EBITDA']
ebitda_resultado_financeiro_cliente = empresa1['EBITDA/Resultado Financeiro']
caixa_divida_cp_cliente = empresa1['Caixa/Divida CP']
roe_cliente = empresa1['ROE']


In [130]:
# Converter as chaves para float
controlador_keys = [float(k) for k in controlador.keys()]
dl_ebitda_keys = [float(k) for k in dl_ebitda.keys()]
margem_ebitda_keys = [float(k) for k in margem_ebitda.keys()]
fco_ebitda_keys = [float(k) for k in fco_ebitda.keys()]
ebitda_resultado_financeiro_keys = [float(k) for k in ebitda_resultado_financeiro.keys()]
caixa_divida_cp_keys = [float(k) for k in caixa_divida_cp.keys()]
roe_keys = [float(k) for k in roe.keys()]

# Encontrar a chave mais próxima
chave_controlador = min(controlador_keys, key=lambda x: abs(x - controlador_cliente))
chave_dl_ebitda = min(dl_ebitda_keys, key=lambda x: abs(x - dl_ebitda_cliente))
chave_margem_ebitda = min(margem_ebitda_keys, key=lambda x: abs(x - margem_ebitda_cliente))
chave_fco_ebitda = min(fco_ebitda_keys, key=lambda x: abs(x - fco_ebitda_cliente))
chave_ebitda_resultado_financeiro = min(ebitda_resultado_financeiro_keys, key=lambda x: abs(x - ebitda_resultado_financeiro_cliente))
chave_caixa_divida_cp = min(caixa_divida_cp_keys, key=lambda x: abs(x - caixa_divida_cp_cliente))
chave_roe = min(roe_keys, key=lambda x: abs(x - roe_cliente))

def ajustar_numero(s):
    try:
        n = float(s)
        if n.is_integer():
            return str(int(n))
        else:
            return s
    except ValueError:
        return s

# Retornar o valor correspondente
resultado_controlador = controlador[str(ajustar_numero(chave_controlador))]
resultado_dl_ebitda = dl_ebitda[str(ajustar_numero(chave_dl_ebitda))]
resultado_margem_ebitda = margem_ebitda[str(ajustar_numero(chave_margem_ebitda))]
resultado_fco_ebitda = fco_ebitda[str(ajustar_numero(chave_fco_ebitda))]
resultado_ebitda_resultado_financeiro = ebitda_resultado_financeiro[str(ajustar_numero(chave_ebitda_resultado_financeiro))]
resultado_caixa_divida_cp = caixa_divida_cp[str(ajustar_numero(chave_caixa_divida_cp))]
resultado_roe = roe[str(ajustar_numero(chave_roe))]

In [131]:
lista_resultados = []
lista_resultados.append(resultado_controlador)
lista_resultados.append(resultado_dl_ebitda)
lista_resultados.append(resultado_margem_ebitda)
lista_resultados.append(resultado_fco_ebitda)
lista_resultados.append(resultado_ebitda_resultado_financeiro)
lista_resultados.append(resultado_caixa_divida_cp)
lista_resultados.append(resultado_roe)

In [132]:
lista_pesos = list(pesos.values())

In [133]:
import numpy as np
resultado = np.dot(lista_resultados, lista_pesos)

In [134]:
score_final = {'1':'AAA','1.5':'AA+','2':'AA','2.5':'AA-','3':'A+','3.5':'A','4':'A-','4.5':'BBB+','5':'BBB'}

In [135]:
score_keys = [float(k) for k in score_final.keys()]
chave_score = min(score_keys, key=lambda x: abs(x - resultado))
resultado_score = score_final[str(ajustar_numero(chave_score))]
nome_empresa = empresa1['Emissor']

In [136]:
from qlabs.database import PostgreSQL
db = PostgreSQL()

In [137]:
ratings = db.get_data("SELECT * FROM compliance.credito_rating")

In [138]:
df_setor = ratings.loc[ratings['setor'] == setor_cliente]

In [139]:
print(f"SCORE ({nome_empresa}): \
\n DL/EBITDA:{resultado_dl_ebitda} \
\n FCO/EBITDA: {resultado_fco_ebitda} \
\n Margem EBITDA: {resultado_margem_ebitda} \
\n EBITDA/Resultado Financeiro: {resultado_ebitda_resultado_financeiro} \
\n Caixa/Dívida CP: {resultado_caixa_divida_cp} \
\n Controlador: {resultado_controlador} \
\n ROE: {resultado_roe} \
\n Score Final: {resultado} \
\n Rating Atribuído: {resultado_score}")

SCORE (TELEFONICA BRASIL): 
 DL/EBITDA:1 
 FCO/EBITDA: 1 
 Margem EBITDA: 3 
 EBITDA/Resultado Financeiro: 1 
 Caixa/Dívida CP: 3 
 Controlador: -1 
 ROE: 4 
 Score Final: 1.8499999999999996 
 Rating Atribuído: AA


In [140]:
lista_valores = []
for index, value in df_setor.iterrows():
    setor_cliente = value['setor']
    controlador_cliente = value['controlador']
    dl_ebitda_cliente = value['dl_ebitda']
    margem_ebitda_cliente = value['margem_ebitda']
    fco_ebitda_cliente = value['fco_ebitda']
    ebitda_resultado_financeiro_cliente = value['ebitda_resultado_financeiro']
    caixa_divida_cp_cliente = value['caixa_divida_cp']
    roe_cliente = value['roe']

    chave_controlador = min(controlador_keys, key=lambda x: abs(x - controlador_cliente))
    chave_dl_ebitda = min(dl_ebitda_keys, key=lambda x: abs(x - dl_ebitda_cliente))
    chave_margem_ebitda = min(margem_ebitda_keys, key=lambda x: abs(x - margem_ebitda_cliente))
    chave_fco_ebitda = min(fco_ebitda_keys, key=lambda x: abs(x - fco_ebitda_cliente))
    chave_ebitda_resultado_financeiro = min(ebitda_resultado_financeiro_keys, key=lambda x: abs(x - ebitda_resultado_financeiro_cliente))
    chave_caixa_divida_cp = min(caixa_divida_cp_keys, key=lambda x: abs(x - caixa_divida_cp_cliente))
    chave_roe = min(roe_keys, key=lambda x: abs(x - roe_cliente))

    resultado_controlador = controlador[str(ajustar_numero(chave_controlador))]
    resultado_dl_ebitda = dl_ebitda[str(ajustar_numero(chave_dl_ebitda))]
    resultado_margem_ebitda = margem_ebitda[str(ajustar_numero(chave_margem_ebitda))]
    resultado_fco_ebitda = fco_ebitda[str(ajustar_numero(chave_fco_ebitda))]
    resultado_ebitda_resultado_financeiro = ebitda_resultado_financeiro[str(ajustar_numero(chave_ebitda_resultado_financeiro))]
    resultado_caixa_divida_cp = caixa_divida_cp[str(ajustar_numero(chave_caixa_divida_cp))]
    resultado_roe = roe[str(ajustar_numero(chave_roe))]

    lista_resultados = []
    lista_resultados.append(resultado_controlador)
    lista_resultados.append(resultado_dl_ebitda)
    lista_resultados.append(resultado_margem_ebitda)
    lista_resultados.append(resultado_fco_ebitda)
    lista_resultados.append(resultado_ebitda_resultado_financeiro)
    lista_resultados.append(resultado_caixa_divida_cp)
    lista_resultados.append(resultado_roe)

    resultado_emp = np.dot(lista_resultados, lista_pesos)
    lista_valores.append(resultado_emp)

In [141]:
from statistics import mean,median
rating_medio = mean(lista_valores)
rating_mediano = median(lista_valores)


In [142]:
chave_setorial_medio = min(score_keys, key=lambda x: abs(x - rating_medio))
resultado_setorial_medio = score_final[str(ajustar_numero(chave_setorial_medio))]
chave_setorial_mediano = min(score_keys, key=lambda x: abs(x - rating_mediano))
resultado_setorial_mediano = score_final[str(ajustar_numero(chave_setorial_mediano))]


In [143]:
def percentil_do_valor(lista, valor):
    lista_ordenada = sorted(lista)
    n = len(lista_ordenada)
    menores_ou_iguais = sum(v <= valor for v in lista_ordenada)
    return menores_ou_iguais / n

percentil = percentil_do_valor(lista_valores, resultado)

In [144]:
print(f"SCORE Setorial(benchmark): \
\n Setor:{setor_cliente} \
\n Média do Setor: {rating_medio} \
\n Mediana do Setor: {rating_mediano} \
\n Rating Médio: {resultado_setorial_medio} \
\n Rating Mediano: {resultado_setorial_mediano} \
\n Percentil: {percentil}")

SCORE Setorial(benchmark): 
 Setor:Telecom 
 Média do Setor: 2.0625 
 Mediana do Setor: 2.025 
 Rating Médio: AA 
 Rating Mediano: AA 
 Percentil: 0.5
